In [1]:
DATA_DIR = '/home/nsuprotivniy/Documents/Работа/OKru/Antispam/data/'
SAMPLE_SIZE = -1

In [2]:
!pip install pyarrow
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import seaborn as sns
import user_agents
%matplotlib inline
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.tree import export_graphviz

from random import *

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
columns=['USER_ID', 'TIMESTAMP', 'LOGIN_CREDENTIAL_TYPE']
botsLogins = pq.read_table(DATA_DIR + 'botsLogins.parquet', columns=columns).to_pandas().head(SAMPLE_SIZE >> 1)
usersLogins = pq.read_table(DATA_DIR + 'usersLogins.parquet', columns=columns).to_pandas().head(SAMPLE_SIZE >> 1)

In [4]:
botsLogins['isBot'] = 1
usersLogins['isBot'] = 0
logins = botsLogins.append(usersLogins)

In [5]:
#logins = logins[logins['LOGIN_CREDENTIAL_TYPE'].isin(['LOGIN')]

In [6]:
# rid = randrange(0, botsLogins['USER_ID'].count())
# ruid = botsLogins.iloc(0)[rid]['USER_ID']
# rlogins = botsLogins[botsLogins['USER_ID'] == ruid]
# plt.figure(figsize=(15, 10), dpi= 70, facecolor='w', edgecolor='k')
# ts_types = ['EMAIL', 'PHONE', 'LOGIN', 'API_TOKEN', 'AUTH_CODE', 'SOCIAL_API_TOKEN', 'JSESSION_ID', 'USER_ID_NO_PASSWORD']
# cmap = plt.get_cmap('viridis')
# colors = cmap(np.linspace(0, 1, len(ts_types)))

# for i in range(0, len(ts_types)):
#     data = rlogins[rlogins['LOGIN_CREDENTIAL_TYPE'] == ts_types[i]]['TIMESTAMP']
#     print(data.count(), ts_types[i])
#     plt.scatter(data, [i] * data.count(), c=colors[i], label=ts_types[i])
# plt.legend();

In [7]:
cred_counts_all = logins.groupby(['USER_ID', 'LOGIN_CREDENTIAL_TYPE']).size().unstack(fill_value=0)
cred_counts_all.mean()

LOGIN_CREDENTIAL_TYPE
API_SESSION            0.000751
API_TOKEN              0.835125
AUTH_CODE              0.138116
EMAIL                  0.006535
JSESSION_ID            0.153228
LOGIN                  0.027239
MESSENGER              0.006939
PHONE                  0.005403
SOCIAL_API_TOKEN       0.000003
TWO_FACTOR             0.000073
USER_ID                0.000016
USER_ID_NO_PASSWORD    0.057862
dtype: float64

In [8]:
cred_counts_bots = botsLogins.groupby(['USER_ID', 'LOGIN_CREDENTIAL_TYPE']).size().unstack(fill_value=0)
cred_counts_bots.mean()

LOGIN_CREDENTIAL_TYPE
API_SESSION            0.024002
API_TOKEN              0.383307
AUTH_CODE              1.963696
EMAIL                  0.909179
JSESSION_ID            9.661320
LOGIN                  6.887710
MESSENGER              0.009046
PHONE                  0.726330
SOCIAL_API_TOKEN       0.000362
TWO_FACTOR             0.000724
USER_ID                0.000482
USER_ID_NO_PASSWORD    0.651067
dtype: float64

In [9]:
cred_counts_users = usersLogins.groupby(['USER_ID', 'LOGIN_CREDENTIAL_TYPE']).size().unstack(fill_value=0)
cred_counts_users.mean()

LOGIN_CREDENTIAL_TYPE
API_SESSION            0.000721
API_TOKEN              0.835698
AUTH_CODE              0.135800
EMAIL                  0.005390
JSESSION_ID            0.141165
LOGIN                  0.018535
MESSENGER              0.006937
PHONE                  0.004488
SOCIAL_API_TOKEN       0.000002
TWO_FACTOR             0.000072
USER_ID                0.000015
USER_ID_NO_PASSWORD    0.057109
dtype: float64

In [10]:
pd.DataFrame({'users': cred_counts_users.mean(), 'bots': cred_counts_bots.mean(), 'all': cred_counts_all.mean()})

,all,bots,users
LOGIN_CREDENTIAL_TYPE,,,
API_SESSION,0.000751,0.024002,0.000721
API_TOKEN,0.835125,0.383307,0.835698
AUTH_CODE,0.138116,1.963696,0.135800
EMAIL,0.006535,0.909179,0.005390
JSESSION_ID,0.153228,9.661320,0.141165
LOGIN,0.027239,6.887710,0.018535
MESSENGER,0.006939,0.009046,0.006937
PHONE,0.005403,0.726330,0.004488
SOCIAL_API_TOKEN,0.000003,0.000362,0.000002


In [11]:
%%time
ts_min = logins['TIMESTAMP'].min()
logins['TS_PART'] = logins['TIMESTAMP'].apply(lambda x: int((x - ts_min) / 60000))

CPU times: user 4.74 s, sys: 380 ms, total: 5.12 s
Wall time: 5.12 s


In [12]:
%%time
ts_part_cred_count = logins.groupby(['TS_PART','USER_ID', 'isBot', 'LOGIN_CREDENTIAL_TYPE']).size().unstack(fill_value=0)
ts_part_cred_count.rename(index=str, columns={"USER_ID": "USER_ID_TYPE"}, inplace=True)
ts_part_cred_count.reset_index(inplace=True)

CPU times: user 1min 24s, sys: 6.51 s, total: 1min 30s
Wall time: 1min 52s


In [13]:
ts_part_cred_count.head()

LOGIN_CREDENTIAL_TYPE,TS_PART,USER_ID,isBot,API_SESSION,API_TOKEN,AUTH_CODE,EMAIL,JSESSION_ID,LOGIN,MESSENGER,PHONE,SOCIAL_API_TOKEN,TWO_FACTOR,USER_ID_TYPE,USER_ID_NO_PASSWORD
0,0,+i+1RrUvgJSBjn81rbahVQ==,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0KDCv6C5Z/kgj8/85vbGLg==,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0QbpukZlig9o975h7t41CQ==,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,17glEQaGl3XOREG0dGknzg==,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,1Ba+CG+zW6QLgPD+XHfTZQ==,1,0,0,1,0,0,0,0,0,0,0,0,0


In [14]:
ts_part_cred_count.count()

LOGIN_CREDENTIAL_TYPE
TS_PART                7994610
USER_ID                7994610
isBot                  7994610
API_SESSION            7994610
API_TOKEN              7994610
AUTH_CODE              7994610
EMAIL                  7994610
JSESSION_ID            7994610
LOGIN                  7994610
MESSENGER              7994610
PHONE                  7994610
SOCIAL_API_TOKEN       7994610
TWO_FACTOR             7994610
USER_ID_TYPE           7994610
USER_ID_NO_PASSWORD    7994610
dtype: int64

In [15]:
# %%time
# ts_part_cred_count.mean()

In [16]:
features = ['API_SESSION', 'API_TOKEN', 'AUTH_CODE',
            'EMAIL', 'JSESSION_ID', 'LOGIN', 'MESSENGER', 'PHONE',
            'SOCIAL_API_TOKEN', 'TWO_FACTOR', 'USER_ID_TYPE',
            'USER_ID_NO_PASSWORD']
X = ts_part_cred_count[features]
y = ts_part_cred_count['isBot'].astype('int64')

In [17]:
forest = RandomForestClassifier()
forest.fit(X, y)
predicted = forest.predict(X)
f1_score(y, predicted)

0.14514540127681894

In [18]:
# DEPTH_RANGE = range(1, 10, 5)
# MAX_FEATURES_RANGE = range(1, len(features), 5)
# forest = RandomForestClassifier(n_estimators=100,
#                                 n_jobs=-1, 
#                                 random_state=17)
# forest_params = {'max_depth': DEPTH_RANGE,'max_features': MAX_FEATURES_RANGE}
# forest_grid = GridSearchCV(forest, forest_params, cv=2, n_jobs=-1, verbose=True, scoring='f1')
# forest_grid.fit(X, y)
# forest_grid.best_params_, forest_grid.best_score_ 